# Symmetric Encryption

In this notebook, we will explore how to achieve confidentiality in communications. Alice will send a message to Bob in the face of the adversary Eve, who can intercept messages (but cannot tamper with them like Mallory.)

We begin by assuming Alice and Bob share a secret key $k$. Before encryption, Alice's message is written in **plaintext** $p$, clear and readable to anyone. She will make use of an **encryption** algorithm on the key and plaintext $E(p,k)$ to generate an unreadable **ciphertext** message $c$. The ciphertext can be freely transmitted. Without the key, the ciphertext should be unreadable by Eve. Bob can then use the **decryption** algorithm and key to decrypt the ciphertext and recover the original plaintext message $D(c,k)$.

## Caesar Cipher

The Caesar Cipher has been around for thousands of years. It is not secure. We will study it to demonstrate cryptography and to understand how it can be attacked.

The key for the Caesar Cipher is a number between 0 and $\|\Sigma\|$, the size of the alphabet. In the case of upper-case English letters, $\|\Sigma\|=26$. The encryption process replaces each letter with the letter k after it in the alphabet. Let's say our key k=5. Every A in the plaintext is replaced by F, and B's would be replaced by G's. The last 5 letters of the alphabet are replaced by the first 5 letters (i.e. it wraps around like the modulo operator.) Y's are replaced with D's and Z's are replaced by E's.

Let's write some code to do this. First, let's look at code to generate the ciphertext alphabet. We can use slicing to get the last part of the alphabet (plaintextAlphabet\[5:\]everything from F to Z) and and the first 5 letters (plaintextAlphabet\[:5\]=A to E). Concatenating those together gives us a version of the alphabet shifted by 5.

In [ ]:
key=5
plaintextAlphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
ciphertextAlphabet=plaintextAlphabet[5:]+plaintextAlphabet[:5]
print(ciphertextAlphabet)

Now, we can build a dictionary that, when indexed with a plaintext letter, will give us its ciphertext replacement. We'll build and empty dictionary and then loop through both alphabets at the same time using the zip function. This will give us a pair of letters p and c that we can then insert into the dictionary as key-value pairs.

In [ ]:
encryptionDict={}
for p,c in zip(plaintextAlphabet,ciphertextAlphabet):
    encryptionDict[p]=c

As you can see, looking up plaintext letters in this dictionary gives us their ciphertext equivalents.

In [ ]:
print("A replaced by",encryptionDict["A"])
print("B replaced by",encryptionDict["B"])
print("X replaced by",encryptionDict["X"])
print("Y replaced by",encryptionDict["Y"])

Now we're ready to do the encryption. Starting with an empty ciphertext string, we can loop through each letter in the plaintext, look up its ciphertext equivalent, and then append that to the ciphertext.

In [ ]:
plaintext="TOPSECRETTELLNOONE"
ciphertext=""
for p in plaintext:
    ciphertext=ciphertext+encryptionDict[p]
print(ciphertext)

Once we've replaced all the plaintext letters with their ciphertext equivalents, we've encrypted the message into ciphertext. Alice can transmit the message to Bob.

Let's put our encrpytion code into a function.

In [ ]:
def encrypt(plaintext,k):
    pAlphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    cAlphabet=pAlphabet[k:]+pAlphabet[:k]
    eDict={}
    for p,c in zip(pAlphabet,cAlphabet):
        eDict[p]=c
    ciphertext=""
    for p in plaintext:
        ciphertext+=eDict[p]
    return ciphertext

The decryption algorithm works almost identically. All we need to do is reverse the role of the two alphabets when building the dictionary. When we look up a ciphertext character, we want its plaintext equivalent.

In [ ]:
def decrypt(ciphertext,k):
    pAlphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    cAlphabet=pAlphabet[k:]+pAlphabet[:k]
    dDict={}
    for p,c in zip(pAlphabet,cAlphabet):
        dDict[c]=p
    plaintext=""
    for p in ciphertext:
        plaintext+=dDict[p]
    return plaintext

k=5
plaintext="TOPSECRETTELLNOONE"
ciphertext=encrypt(plaintext,k)
print("Encrypted message: "+ciphertext)
decrypted=decrypt(ciphertext,k)
print("Decrypted message: "+decrypted)

## Attacking Caesar Cipher

Now that we understand how the Caesar cipher works, let's examine some of its weaknesses.

### Brute Force

There are only 26 possible keys for the Caesar Cipher. We can brute force attack all 26 keys with a simple loop.

In [ ]:
secret="YTLMUTEELIXVBTE"
for k in range(26):
    print(decrypt(secret,k))

### Cryptanalysis

The small keyspace of the Caesar Cipher is a weakness, but there is another weakness we can attack. The encryption algorithm doesn't change the statistical patterns of the plaintext. Some letters are used more frequently than others. Rather than trying all of the keys, we can count how often each character is used in a bit of ciphertext. We can do this in Python using a dictionary. The keys are the letters and the values are the number of times we observe each letter.

In [ ]:
ciphertext='HPESPAPZAWPZQESPFYTEPODELEPDTYZCOPCEZQZCXLXZCPAPCQPNEFYTZYPDELMWTDSUFDETNPTYDFCPOZXPDETNECLYBFTWTEJACZGTOPQZCESPNZXXZYOPQPYNPACZXZEPESPRPYPCLWHPWQLCPLYODPNFCPESPMWPDDTYRDZQWTMPCEJEZZFCDPWGPDLYOZFCAZDEPCTEJOZZCOLTYLYOPDELMWTDSESTDNZYDETEFETZYQZCESPFYTEPODELEPDZQLXPCTNL'
alphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
counts={}
for c in alphabet:
    counts[c]=0

for c in m:
    counts[c]+=1
    
for c in alphabet:
    print(c,counts[c])

The most frequent letter in the ciphertext is P. If we guess that corresponds to E, that would make the key 11. Decrypting the message using our guess produces the correct plaintext.

In [ ]:
decrypt(ciphertext,11)

## Vigenere Cipher

One way to improve on the Caesar Cipher is to use more than one of them at a time. For example, we could use 3 different keys, and encrypt every third character with a different Caesar Cipher. This helps us with **both** weaknesses described above. Each new key we add dramatically increases the number of possible keys. For n Caesar Cipher keys, we have $26^n$ possible keys. With one, there were only 26 possibilities. With 3, there are 17,576 possibilities.

Let's code up the Vigenere Cipher encryption algorithm. Our key will be a list of Caesar Cipher keys. We will need to prepare a list of ciphertext alphabets (one for each key.) These quirks aside, the rest of the set up code is identical to the Caesar Cipher.

In [ ]:
keys=[5,19,4]

pAlphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
cAlphabets=[]
n=len(keys)
for i in range(n):
    k=keys[i]
    cAlphabet=pAlphabet[k:]+pAlphabet[:k]
    cAlphabets.append(cAlphabet)

Now we need a dictionary for each alphabet.

In [ ]:
eDicts=[]
for i in range(n):
    cAlphabet=cAlphabets[i]
    eDict={}
    for p,c in zip(pAlphabet,cAlphabet):
        eDict[p]=c
    eDicts.append(eDict)

Now we can encrypt our plaintext, making sure to use the right dictionary for each letter.

In [ ]:
plaintext="THENIGHTISDARKANDFULLOFTERRORS"
ciphertext=""
for i,p in enumerate(plaintext):
    ciphertext+=eDicts[i%n][p]

print(ciphertext)

We can put all this code together in a function, and write another to decrypt. As with the Caesar Cipher, all we need to do is change the role of the plaintext and ciphertext characters in the dictionaries to decrypt.

In [ ]:
def vigEncrypt(plaintext,keys):
    pAlphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    cAlphabets=[]
    n=len(keys)
    for i in range(n):
        k=keys[i]
        cAlphabet=pAlphabet[k:]+pAlphabet[:k]
        cAlphabets.append(cAlphabet)
    eDicts=[]
    for i in range(n):
        cAlphabet=cAlphabets[i]
        eDict={}
        for p,c in zip(pAlphabet,cAlphabet):
            eDict[p]=c
        eDicts.append(eDict)
    ciphertext=""
    for i,p in enumerate(plaintext):
        ciphertext+=eDicts[i%n][p]
    return ciphertext

def vigDecrypt(ciphertext,keys):
    pAlphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    cAlphabets=[]
    n=len(keys)
    for i in range(n):
        k=keys[i]
        cAlphabet=pAlphabet[k:]+pAlphabet[:k]
        cAlphabets.append(cAlphabet)
    dDicts=[]
    for i in range(n):
        cAlphabet=cAlphabets[i]
        dDict={}
        for p,c in zip(pAlphabet,cAlphabet):
            dDict[c]=p
        dDicts.append(dDict)
    plaintext=""
    for i,c in enumerate(ciphertext):
        plaintext+=dDicts[i%n][c]
    return plaintext

vigEncrypt("THENIGHTISDARKANDFULLOFTERRORS",[5,4,2,3,1])
vigDecrypt("YLGQJLLVLTIETNBSHHXMQSHWFWVQUT",[5,4,2,3,1])

## One-Time Pads

If we use a Vigenere cipher whose key is as long as our plaintext, there is no remaining statistical information from the plaintext left in the ciphertext. This is known as a **one-time pad**, and it's an unbreakable cryptosystem. As long as our key is truly random, it is impossible to recover the plaintext without the key.

Here is a 26-letter long message I encrypted using a truly random 26 number key.

In [ ]:
ciphertext="XLQOEPTZQVGSZPUYXDBPDVAMTV"

How could you possibly decrypt it? It could be any 26-letter message. Even if you guessed correctly, how would you know if you were right? You have no reference point.

## One-Time Pad in Binary

It's easy enough to extrapolate what we've learned so far into binary. Our alphabet $\Sigma$ in binary is 0 and 1. The keys in binary can be either 0 or 1. We add the key to the letter and compute the result modulo 2.

This computation is equivalent to XORing the plaintext bit with the key bit. If you doubt this, try working through the possibilities yourself.

Here is an example one-time pad encryption using binary.

In [ ]:
k=0b01000010110101001100111010101001
p=0b11011001100110110010010100111000
c=k^p
bin(c)

How do we decrypt? XOR is its own inverse. We just XOR the cipertext with the key to recover the plaintext.

In [ ]:
k=0b01000010110101001100111010101001
c=0b10011011010011111110101110010001
d=k^c
bin(d)

## Stream Ciphers

If generating a stream of random bits gives us secure encryption, what about using a pseudorandom number generator? This encryption method is called a **stream cipher**. The security of a stream cipher depends on the strength of the pseudorandom number generator (i.e. how indistinguishable is it from true randomness) and the strength of the key. Key? What key? 

The key in a stream cipher is the seed for the pseudorandom generator. By seeding the pseudorandom number generator with the same key, a receiver (or attacker) can generate the same pseudorandom stream used to encrypt the plaintext.

We'll use Python's random library to demonstrate these concepts. This library is not cryptographically secure, so we shouldn't use this in practice. This is purely an educational demo.

We use the random.seed() function to seed Python's pseudorandom number generator. We use the random.getrandbits() function to generate an 8-bit pseudorandom numbers. We XOR each of those with the letters in the input.

In [ ]:
import random

def streamCipher(inputString,key):
    random.seed(key)
    returnString=""
    for c in inputString:
        r=random.getrandbits(8)
        returnString+=chr(ord(c)^r)
    return returnString

plaintext="I AM IRON MAN"
ciphertext=streamCipher(plaintext,22)
print(ciphertext)

Why did I use variable names like "inputString" and "returnString" instead of "plaintext" and "ciphertext"?

In [ ]:
decrypted=streamCipher(ciphertext,22)
print(decrypted)

For stream ciphers, the encryption and decryption function is the same!

## Exercises
Below are the exercises for this lesson. Don't forget to take breaks and ask for help if you get stuck.

1) This message was encrypted using the Caesar Cipher with the key 9. Decrypt it. RDWMNABCXXMCQJCANONANWLN

In [ ]:
ciphertext="RDWMNABCXXMCQJCANONANWLN"

2) This message was encrypted using the Caesar Cipher. Decrypt it **without using the key**. CUGALYUNUNVIUNM

In [ ]:
ciphertext="CUGALYUNUNVIUNM"

3) This message was encrypted using the Viginere Cipher with key \[9,23,21,15\]. Decrypt it. XFOWRPDHWLOXVBADAPOPWADCPXMDDKY

In [ ]:
ciphertext="XFOWRPDHWLOXVBADAPOPWADCPXMDDKY"
key=[9,23,21,15]

4) If we're using a one-time pad to encrypt plaintext 0xFFAAEEBB with key 0x33114422, what is the ciphertext?

In [ ]:
p=0xFFAAEEBB
k=0x33114422

5) This ciphertext was encrypted using our stream cipher above. The key is 34. Decrypt it.

In [ ]:
ciphertext='Î{ÂF\x99nÆS®C\t¡\xa0^L¤\x0b\x18HÙÇònV\x12\x17¼D\x8fi\x8fÕW\x98]\r'
streamCipher(ciphertext,34)

6) This ciphertext was encrypted using our stream cipher above. The key is 12-bits long, and the plaintext is English. Can you break it?

In [ ]:
ciphertext="""²é ØUÁi¹wÜ°¢Ð±JºÏ3ÅrC]úëcÆçÉì¦¢R¨ØbôûáHx~Ý¾nk?¶8Ü+N«Ø7¸?ùájÌÙºt'¯b©M"	QL°tÕ­.ø­É£(³i¶KÂkÎ%Å"""


HINT: Manually looking through every possible decrypted message is a huge waste of your time. Don't print every possible output. The text is English. Check to see if common words are in the plaintext first. The example code below might be helpful.

In [ ]:
"the" in "There were some countries where Teen Wolf was released after Back to the Future and so they retitled it The Boy from the Future."